In [ ]:
import os
import shutil

import fiona
import geopandas as gpd
import numpy as np
import pandas as pd
from seabeepy.config import SETTINGS

import seabeepy as sb

In [ ]:
# Login to MinIO
minio_client = sb.storage.minio_login(
    user=SETTINGS.MINIO_ACCESS_ID, password=SETTINGS.MINIO_SECRET_KEY
)

# Convert SeaBee annotation between versions

This notebook makes it possible to convert SeaBee habitats annotation between different versions of the habitat classes. The main aim is to make it possible to re-use old annotation for training new models.

The code here converts class codes in old annotation files to the codes used in newer files (or vice versa), but **only for class names that can be matched between files**. For each level in the old scheme, this code looks for a class at the same level with the same name in the new scheme. If a single matching class can be identified, the class code from the new scheme in applied; otherwise a warning message is printed.

# 1. User input

In [ ]:
# Campaign folder. Should be a folder within '/niva/tidy/annotation'
camp_fold = r"/home/notebook/shared-seabee-ns9879k/niva-tidy/annotation/remoy_2022"

# Version to convert from
from_version = "0-1"

# Version to convert to
to_version = "1-1"

# Folder for saving intermediates
temp_fold = r"/home/notebook/annotation_temp"

## 2. Convert annotation

In [ ]:
# Check user input
from_version = from_version.replace(".", "-")
to_version = to_version.replace(".", "-")

if not os.path.isdir(camp_fold):
    raise FileNotFoundError(f"Folder '{camp_fold}' does not exist.")

# Check annotation already exists for 'from_version'
camp_name = os.path.basename(camp_fold)
gpkg_name = f"{camp_name}_annotation.gpkg"
gpkg_path = os.path.join(camp_fold, gpkg_name)
from_anno_name = f"{camp_name}_merged_annotation_v{from_version}"
layers = fiona.listlayers(gpkg_path)
if from_anno_name not in layers:
    raise KeyError(
        f"Existing annotation not found for v{from_version}: no layer named '{from_anno_name}' in '{gpkg_name}'."
    )

In [ ]:
# Read original annotation
gdf = gpd.read_file(gpkg_path, layer=from_anno_name)
orig_cols = gdf.columns
gdf.rename(
    columns={
        "lev1_code": "lev1_code_old",
        "lev2_code": "lev2_code_old",
        "lev3_code": "lev3_code_old",
    },
    inplace=True,
)

# Get new codes
codes_df = sb.anno.get_class_codes(to_version)
del codes_df["desc"], codes_df["colour"]

# Loop over levels
for level in [1, 2, 3]:
    code_len = level * 2
    lev_df = codes_df[codes_df["code"].str.len() == code_len].drop_duplicates()
    lev_df.rename(
        columns={"code": f"lev{level}_code", "name": f"lev{level}_name"}, inplace=True
    )
    gdf = gdf.merge(lev_df, how="left", on=f"lev{level}_name")

# Find classes with no match in the new scheme
cols = [
    "subarea_id",
    "lev1_code_old",
    "lev2_code_old",
    "lev3_code_old",
    "lev1_name",
    "lev2_name",
    "lev3_name",
    "lev3_desc",
    "lev1_code",
    "lev2_code",
    "lev3_code",
]
unmatched_gdf = gdf[gdf["lev1_code"].isna()][cols].copy()
gdf.dropna(subset="lev1_code", inplace=True)
print(
    "The following classes in the original cannot be matched at all in the new class schema:"
)
display(unmatched_gdf)

# Update names that cannot be matched at lower levels
gdf.loc[gdf["lev2_code"].isna(), "lev2_name"] = np.nan
gdf.loc[gdf["lev3_code"].isna(), "lev3_name"] = np.nan

# Pad gaps in codes for levels 2 and 3
gdf["lev2_code"] = gdf.apply(
    lambda row: (
        row["lev1_code"] + "--" if pd.isna(row["lev2_code"]) else row["lev2_code"]
    ),
    axis=1,
)
gdf["lev3_code"] = gdf.apply(
    lambda row: (
        row["lev2_code"] + "--" if pd.isna(row["lev3_code"]) else row["lev3_code"]
    ),
    axis=1,
)

# Standardise
gdf = gdf[orig_cols]

## 3. Save converted annotation

In [ ]:
# Save locally
conv_gpkg_name = f"{camp_name}_annotation_converted.gpkg"
temp_gpkg = os.path.join(temp_fold, conv_gpkg_name)
layer = f"{camp_name}_merged_annotation_v{to_version}converted"
gdf.to_file(temp_gpkg, layer=layer, driver="GPKG", index=False)

# Copy MinIO
conv_gpkg_path = os.path.join(camp_fold, conv_gpkg_name)
sb.storage.copy_file(
    temp_gpkg,
    conv_gpkg_path,
    minio_client,
    overwrite=False,
)
os.remove(temp_gpkg)

## 4. Publish converted annotation

In [ ]:
# # Define layer styling using SeaBee standard SLDs.
# # 'style_level' is the level in the hierarchy used for colouring polygons
# style_level = 2
# style_dict = {
#     f"{camp_name}_merged_annotation_v{to_version}converted": f"annotation_classes_v{to_version}_level{style_level}.sld",
# }

# # Upload to GeoServer
# store_name = sb.geo.upload_geopackage_layers_to_geoserver(
#     conv_gpkg_path,
#     [layer],
#     SETTINGS.GEOSERVER_USER,
#     SETTINGS.GEOSERVER_PASSWORD,
#     workspace="geonode",
#     style_dict=style_dict,
# )

# # Publish to GeoNode
# sb.geo.publish_to_geonode(
#     layer,
#     SETTINGS.GEONODE_USER,
#     SETTINGS.GEONODE_PASSWORD,
#     store_name=store_name,
#     workspace="geonode",
# )